In [2]:
import os
import uuid
import shutil
#import pylatex
import pandas as pd
import lightkurve as lk 
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from datetime import datetime
from tqdm.notebook import tqdm
from multiprocessing import cpu_count, Pool
#from pylatex.section import Chapter, Subsection
#from pylatex import Document, Section, Figure, NoEscape, Command

%run pipeline.ipynb

import warnings
warnings.filterwarnings("ignore")

In [3]:
bestlcs=pd.read_csv('makingtable/bestlcs.csv')
gaiatable=pd.read_csv('makingtable/cleanedgaiadata.csv')

In [4]:
lclist=np.array([], dtype=object)

In [5]:
start_col=bestlcs.columns.get_loc('Sector 1')

In [6]:
for index, row in bestlcs.iterrows(): #iterate through each object
    for col in bestlcs.columns[start_col:]: #iterate through all the columns starting with the sectors
        value=row[col]
        if not pd.isna(value):
            lclist=np.append(lclist, [f'{row['pop_id']}_{value}'])

In [7]:
# for curstring in lclist[:10]: #starting with only 10 for testing
#     parts=curstring.split("_")
#     pop_id=parts[0]
#     lcname=parts[1]
#     curlc=get_lc_string(lcname)
#     fig=multiplot3(curlc,lcname,gaiatable)

#     if not os.path.isdir(f'multiplots/{pop_id}/'):
#         os.mkdir(f'multiplots/{pop_id}/')
        
#     fig.savefig(f'multiplots/{pop_id}/{lcname}.png',bbox_inches='tight')

In [8]:
for curstring in lclist: 
    parts=curstring.split("_")
    pop_id=parts[0]
    lcname=parts[1]
    
    if os.path.isfile(f'multiplots/{pop_id}/{lcname}.png'): #don't replot ones that already have been plotted
        continue

    curlc=get_lc_string(lcname)
    try: #make sure that an error with plotting one doesn't break the loop
        fig=multiplot3(curlc,lcname,gaiatable)
    
        if not os.path.isdir(f'multiplots/{pop_id}/'): #check if folder exists and if not make it
            os.mkdir(f'multiplots/{pop_id}/')
            
        fig.savefig(f'multiplots/{pop_id}/{lcname}.png',bbox_inches='tight')
    except Exception as e:
        print(f'Error processing {lcname}: {e}')

Error processing TIC-404335106-38-FFI-10min: index -1 is out of bounds for axis 0 with size 0
Error processing TIC-127866051-11-FFI-30min: index 0 is out of bounds for axis 0 with size 0
Error processing TIC-127866051-38-FFI-10min: index 0 is out of bounds for axis 0 with size 0
Error processing TIC-127866051-65-FFI-200s: index 0 is out of bounds for axis 0 with size 0
Error processing TIC-300651846-13-FFI-30min: index -1 is out of bounds for axis 0 with size 0
Error processing TIC-442571495-91-FFI-200s: index -1 is out of bounds for axis 0 with size 0
Error processing TIC-12779416-91-FFI-200s: index 0 is out of bounds for axis 0 with size 0


Error processing EPIC-204364515-2-EVEREST-1800: index 0 is out of bounds for axis 0 with size 0
Error processing EPIC-204364515-15-EVEREST-1800: index 0 is out of bounds for axis 0 with size 0
Error processing TIC-49040871-91-FFI-200s: index 0 is out of bounds for axis 0 with size 0


Error processing EPIC-205046529-2-EVEREST-1800: index 0 is out of bounds for axis 0 with size 0


Error processing TIC-442575691-91-FFI-200s: index -1 is out of bounds for axis 0 with size 0
Error processing TIC-244161191-42-FFI-10min: index -1 is out of bounds for axis 0 with size 0
